In [0]:
import json
import requests
from pyspark.sql.functions import explode, col, split, trim, expr
import pandas as pd

In [0]:
# Step 1: Make an API request
url = "https://olinda.bcb.gov.br/olinda/servico/MPV_DadosAbertos/versao/v1/odata/MeiosdePagamentosMensalDA(AnoMes=@AnoMes)?@AnoMes='202401'&$top=10000000&$format=json&$select=AnoMes,quantidadePix,valorPix,quantidadeTED,valorTED,quantidadeTEC,valorTEC,quantidadeCheque,valorCheque,quantidadeBoleto,valorBoleto,quantidadeDOC,valorDOC"  # Replace with your API endpoint


response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Step 2: Normalize the JSON response
    data = response.json()
    normalized_data = pd.json_normalize(data['value'], sep = ',')
    # Step 3: Convert to pandas DataFrame
    df = pd.DataFrame(normalized_data)
else:
    print(f"Error: {response.status_code}")
    print(response.text)


In [0]:
df.display()

AnoMes,quantidadePix,valorPix,quantidadeTED,valorTED,quantidadeTEC,valorTEC,quantidadeCheque,valorCheque,quantidadeBoleto,valorBoleto,quantidadeDOC,valorDOC
202405,5229671.82,2137438.65,69023.11,3482036.97,0.0,0.0,10467.85,40133.55,349180.92,506302.82,0.0,0.0
202404,4924640.69,2083741.65,70647.15,3652606.48,0.0,0.0,11179.22,43295.4,361633.28,524619.69,0.0,0.0
202403,4903340.45,1936144.86,67565.05,3333054.27,0.0,0.0,10122.46,36644.55,336435.46,469047.59,0.0,0.0
202402,4367128.53,1718214.16,65805.87,2981226.45,60.01,109.32,10373.38,37076.26,332269.47,463066.01,61.89,34.21
202401,4371542.0,1818473.97,70343.25,3337344.85,387.18,815.94,11919.12,41615.89,363550.32,516702.44,600.76,616.58


In [0]:
df = spark.createDataFrame(df)

In [0]:
df.repartition(20)\
              .write.format("delta")\
              .mode("overwrite")\
              .partitionBy('AnoMes')\
              .option("overwriteSchema", "true")\
              .save("/dbfs/meiospagamento")